In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
df_bohdan = pd.read_csv('final_dataset.csv')
df_sofiia = pd.read_csv('rdi_lonlat.csv')
df_names = pd.read_csv('archive-diploma-kaggle/all_dist_villages.csv')

In [ ]:
df_names = df_names[["admin4Name", "admin4Na_1", "admin4Pcod"]]

In [ ]:
df_orig = df_bohdan.merge(df_sofiia, on='admin4Pcod', how='inner', suffixes=('_1', ''))
df_orig = df_orig.merge(df_names, on='admin4Pcod', how='inner', suffixes=('_1', ''))

In [ ]:
df = df_orig[['lon', 'lat', 'quality', 'admin4Name', "admin4Pcod"]]
df = df.rename(columns={"admin4Pcod": "Postal code", "admin4Name": "Locality", 'quality': "RDL"})
df["admin4Pcod"] = df["Postal code"]

df['RDL'] = np.round(df['RDL'], 3)

bins = [0, 0.15, 0.30, 0.50, 0.70, 1.00]
labels = ['Very High', 'High', 'Moderate', 'Low', 'Very Low']
labels_legend = ['0.00-0.15 (Very High)', '0.15-0.30 (High)', '0.30-0.50 (Moderate)', '0.50-0.70 (Low)', '0.70-1.00 (Very Low)']
df['RDL category'] = pd.cut(df['RDL'], bins=bins, labels=labels, include_lowest=True)
df['RDL legend'] = pd.cut(df['RDL'], bins=bins, labels=labels_legend, include_lowest=True)
df.sort_values('RDL', ascending=True, inplace=True)

custom_data = df[['Locality', "Postal code", 'RDL']]

In [ ]:
import geopandas as gpd
import pandas as pd

# Load the shapefile
shp_file = r"rdi1\rdi1.shp"
gdf = gpd.read_file(shp_file)

# Ensure 'admin4Pcod' is the common column in both dataframes
gdf = gdf.merge(df[['admin4Pcod', 'RDL']], on='admin4Pcod', how='left')

# Save to a new shapefile with the added column
gdf.to_file("graph_model.shp")

In [ ]:
center = (0.5 * (df['lon'].max() + df['lon'].min()), 0.5 * (df['lat'].max() + df['lat'].min()))
print(center)

In [ ]:
colors = { '0.70-1.00 (Very Low)': '#d7191c', '0.50-0.70 (Low)': '#f5a153', '0.30-0.50 (Moderate)': '#f7ff0a', '0.15-0.30 (High)': '#8acc62', '0.00-0.15 (Very High)': '#1a9641' }

fig = px.scatter_mapbox(df, lon='lon', lat='lat', color='RDL legend', color_discrete_map=colors, hover_data={
        # 'lon': False,  # Hide X
        # 'lat': False,  # Hide Y
        'RDL category': True,  # Hide RDI category
        'Locality': True,  # Show RDI (continuous data)
        'Postal code': True,  # Show Category
        'RDL': True  # Show Description
    },
    mapbox_style='open-street-map',
    zoom=5,
    center={
        'lon': center[0],
        'lat': center[1]
    },)
    # width=976,
    # height=686)

hovertemplate = "<br>".join([
    "<b>%{customdata[1]}</b>",
    "%{customdata[2]}",
    "RDI: %{customdata[3]} (%{customdata[0]})<extra></extra>",
])

fig.update_traces(
    marker=dict(size=6),
    hovertemplate=hovertemplate,
    # showlegend=False,
)

fig.update_layout(
    # title='Sample scatter plot',
    plot_bgcolor="white",
    xaxis_title=None,
    xaxis_showticklabels=False,
    yaxis_title=None,
    yaxis_showticklabels=False,
    legend=dict(
        title_text='Rural development level',
        title_font=dict(size=16, color='black'),
        x=0.1,  # Position x at the left
        y=0.1,  # Position y at the bottom
        xanchor='left',  # Anchor the x position to the left
        yanchor='bottom',  # Anchor the y position to the bottom
        orientation='v',
        bgcolor='rgba(255, 255, 255, 0)',  # Transparent background
        bordercolor='rgba(255, 255, 255, 0)',  # Optional: remove border
        itemsizing='constant'
    ),
)

fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig['layout'].update(margin=dict(l=0,r=0,b=0,t=24))

config = {'scrollZoom': True}

fig.show(config=config)

fig.write_html('rdl_graph_interactive.html', config=config)